In [1]:
import numpy as np
import pandas as pd
#dataset import
df = pd.read_csv("../Data/data_5_nodes_yen.csv") #You need to change #directory accordingly
df.head()

,"X(1, 2)","X(1, 3)","X(1, 4)","X(2, 4)","X(2, 5)","X(3, 4)","X(3, 5)","X(4, 5)","Y1(1, 2)","Y1(1, 3)",...,"Y3(3, 5)","Y3(4, 5)","Y4(1, 2)","Y4(1, 3)","Y4(1, 4)","Y4(2, 4)","Y4(2, 5)","Y4(3, 4)","Y4(3, 5)","Y4(4, 5)"
0,40,30,50,40,90,90,70,70,0,1,...,0,0,1,0,0,1,0,0,0,1
1,10,30,20,60,70,90,80,80,1,0,...,1,0,1,0,0,1,0,0,0,1
2,50,60,70,80,90,90,50,10,0,0,...,0,0,1,0,0,1,0,0,0,1
3,30,70,20,10,70,20,70,60,0,0,...,0,0,1,0,0,1,0,0,0,1
4,60,80,10,10,20,40,60,60,0,0,...,0,0,0,0,1,0,0,1,1,0


In [2]:
edges = [[1, 2], [1, 3], [1, 4], [2, 4], [2, 5], [3, 4], [3, 5], [4, 5]]

In [3]:
number_edges = 8
number_steps = 4
start_node = 1
end_node = 5

In [4]:
X = df.iloc[:50000,:number_edges].values
y = df.iloc[:50000,number_edges:].values

In [5]:
X.shape

(50000, 8)

In [6]:
y.shape

(50000, 32)

In [7]:
from sklearn.model_selection import train_test_split
X1,X2,y_train,y_test = train_test_split(X,y,test_size = 0.1)

In [8]:
X2[1]

array([20, 60, 90, 40, 60, 60, 40, 40], dtype=int64)

In [9]:
y_test[1]

array([1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1], dtype=int64)

In [10]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X1)
X_test= sc.fit_transform(X2)

In [11]:
X_train = np.repeat(X_train[:, np.newaxis, : ], number_steps, axis=1)
X_train.shape

(45000, 4, 8)

In [12]:
X_test = np.repeat(X_test[:, np.newaxis, : ], number_steps, axis=1)
X_test.shape

(5000, 4, 8)

In [13]:
y_train[0]

array([0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1], dtype=int64)

In [14]:
y_train_2 = np.reshape(y_train,(y_train.shape[0], number_steps, number_edges))
y_train_2.shape

(45000, 4, 8)

In [15]:
y_train_2[0]

array([[0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [16]:
y_test_2 = np.reshape(y_test,(y_test.shape[0], number_steps, number_edges))
y_test_2.shape

(5000, 4, 8)

In [17]:
X2[1,:]

array([20, 60, 90, 40, 60, 60, 40, 40], dtype=int64)

In [18]:
#Dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,TimeDistributed,SimpleRNN
# Neural network
model = Sequential()
model.add(SimpleRNN(32, input_shape=(number_steps, number_edges), 
                    return_sequences=True))
model.add(SimpleRNN(16, return_sequences=True))
model.add(TimeDistributed(Dense(number_edges, activation='sigmoid')))

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 32)             1312      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 4, 16)             784       
                                                                 
 time_distributed (TimeDistr  (None, 4, 8)             136       
 ibuted)                                                         
                                                                 
Total params: 2,232
Trainable params: 2,232
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.compile(loss='BinaryCrossentropy', optimizer='adam')

In [21]:
history = model.fit(X_train, y_train_2, epochs=1000, batch_size=64)

Epoch 1/1000
704/704 [==============================] - 3s 2ms/step - loss: 0.4980
Epoch 2/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.3627
Epoch 3/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.3141
Epoch 4/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.2833
Epoch 5/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.2600
Epoch 6/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.2394
Epoch 7/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.2224
Epoch 8/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.2093
Epoch 9/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.1984
Epoch 10/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.1895
Epoch 11/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.1822
Epoch 12/1000
704/704 [==============================] - 2s 3ms/step - loss: 0.1760
E

704/704 [==============================] - 2s 2ms/step - loss: 0.0765
Epoch 99/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0761
Epoch 100/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0760
Epoch 101/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0754
Epoch 102/1000
704/704 [==============================] - 1s 2ms/step - loss: 0.0753
Epoch 103/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0745
Epoch 104/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0751
Epoch 105/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0740
Epoch 106/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0739
Epoch 107/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0740
Epoch 108/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0735
Epoch 109/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0

704/704 [==============================] - 2s 2ms/step - loss: 0.0617
Epoch 195/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0621
Epoch 196/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0618
Epoch 197/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0619
Epoch 198/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0618
Epoch 199/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0614
Epoch 200/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0615
Epoch 201/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0619
Epoch 202/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0616
Epoch 203/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0616
Epoch 204/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0613
Epoch 205/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.

704/704 [==============================] - 2s 2ms/step - loss: 0.0566
Epoch 291/1000
704/704 [==============================] - 1s 2ms/step - loss: 0.0567
Epoch 292/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0568
Epoch 293/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0570
Epoch 294/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0566
Epoch 295/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0562
Epoch 296/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0562
Epoch 297/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0566
Epoch 298/1000
704/704 [==============================] - 1s 2ms/step - loss: 0.0564
Epoch 299/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0568
Epoch 300/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.0568
Epoch 301/1000
704/704 [==============================] - 2s 2ms/step - loss: 0.

KeyboardInterrupt: 

In [22]:
len(model.layers)

3

In [23]:
pred=model.predict(X_train)

1407/1407 [==============================] - 2s 1ms/step


In [24]:
pred.shape

(45000, 4, 8)

In [25]:
pred[0]

array([[1.09657450e-02, 1.31454159e-04, 9.96019006e-01, 1.40939796e-04,
        2.15834379e-03, 2.61644054e-05, 2.50163634e-04, 9.96446013e-01],
       [9.99954164e-01, 1.78521859e-05, 1.81041614e-05, 8.51011373e-06,
        9.99938548e-01, 1.40496477e-05, 9.57452175e-06, 3.95865354e-05],
       [1.60914436e-01, 8.39555085e-01, 1.38946844e-03, 2.74963111e-01,
        1.57196913e-03, 1.35315233e-03, 8.39492142e-01, 9.83050093e-02],
       [9.92056668e-01, 1.07908294e-04, 4.11912799e-03, 9.93004501e-01,
        7.41930344e-05, 3.33406636e-03, 3.07086878e-03, 9.98699129e-01]],
      dtype=float32)

In [26]:
y_train_2[0]

array([[0, 0, 1, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 1, 0, 0, 0, 1]], dtype=int64)

In [27]:
import copy
def cost_path(x,y):
    cost = 0
    path = []
    for i in range(number_edges):
        if y[i]==1:
            path.append(edges[i])
            cost+= x[i]
    if(len(path)!=0):
        path_2 = copy.deepcopy(path)
        path_nodes = [start_node]
        if(path[0][0]!=start_node):
            cost = 0
        else:
            path_nodes.append(path[0][1])
            del path_2[0]
            condition = True
            while(len(path_2)!=0 and condition):
                condition = False
                for i in range(len(path_2)):
                    if(path_nodes[-1] in path_2[i]):
                        path_2[i].remove(path_nodes[-1])
                        path_nodes.append(path_2[i][0])
                        path_2.pop(i)
                        condition = True
                        break
        if(path_nodes[-1]!=end_node or len(path_2)!=0):
            cost = 0
    else:
        cost = 0
    return cost

In [28]:
cost_path(X1[1],y_train[1])

50

In [29]:
accuracy = np.zeros(number_steps+1)
for i in range(pred.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_train_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy[t] += 1
    if condition_1:
        accuracy[number_steps] += 1
accuracy = np.round(accuracy / pred.shape[0] * 100,2)
accuracy

array([96.62, 93.87, 94.45, 90.8 , 81.95])

In [30]:
pred_test=model.predict(X_test)

157/157 [==============================] - 0s 1ms/step


In [31]:
accuracy_test = np.zeros(number_steps+1)
for i in range(pred_test.shape[0]):
    condition_1 = True
    for t in range(number_steps):
        output = copy.deepcopy(pred_test[i][t])
        condition_2 = True
        for j in range(len(output)):
            output[j] = round(output[j])
        if(cost_path(X1[i],output) != cost_path(X1[i],y_test_2[i][t])):
            condition_2 = False
            condition_1 = False
        if condition_2:
            accuracy_test[t] += 1
    if condition_1:
        accuracy_test[number_steps] += 1
accuracy_test = np.round(accuracy_test / pred_test.shape[0] * 100,2)
accuracy_test

array([96.36, 92.46, 92.86, 88.46, 79.14])

In [32]:
model.summary()
print(accuracy)
print(accuracy_test)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 4, 32)             1312      
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 4, 16)             784       
                                                                 
 time_distributed (TimeDistr  (None, 4, 8)             136       
 ibuted)                                                         
                                                                 
Total params: 2,232
Trainable params: 2,232
Non-trainable params: 0
_________________________________________________________________
[96.62 93.87 94.45 90.8  81.95]
[96.36 92.46 92.86 88.46 79.14]


In [33]:
import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for loss
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

NameError: name 'history' is not defined